<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 4.1.2 Linear Regression

## Prediction of House Price Using Linear Regression

### Data

The Sacramento real estate transactions file is a list of 985 real estate transactions in the Sacramento area reported over a five-day period, as reported by the Sacramento Bee.

In [2]:
import pandas as pd
import numpy as np
import pandas as pd

%matplotlib inline

### 1. Explore Data

Perform EDA on this dataset. Find Dependent and Independent Variables to predict house price.

In [3]:
# Read CSV
house_csv = pd.read_csv("Sacramento_transactions.csv")

In [4]:
house_csv.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [5]:
house_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   street     985 non-null    object 
 1   city       985 non-null    object 
 2   zip        985 non-null    int64  
 3   state      985 non-null    object 
 4   beds       985 non-null    int64  
 5   baths      985 non-null    int64  
 6   sq__ft     985 non-null    int64  
 7   type       985 non-null    object 
 8   sale_date  985 non-null    object 
 9   price      985 non-null    int64  
 10  latitude   985 non-null    float64
 11  longitude  985 non-null    float64
dtypes: float64(2), int64(5), object(5)
memory usage: 92.5+ KB


In [6]:
#drop rows with '0'
house_csv = house_csv[(house_csv[['beds','baths', 'sq__ft']] != 0).all(axis=1)]
house_csv

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768
...,...,...,...,...,...,...,...,...,...,...,...,...
980,9169 GARLINGTON CT,SACRAMENTO,95829,CA,4,3,2280,Residential,Thu May 15 00:00:00 EDT 2008,232425,38.457679,-121.359620
981,6932 RUSKUT WAY,SACRAMENTO,95823,CA,3,2,1477,Residential,Thu May 15 00:00:00 EDT 2008,234000,38.499893,-121.458890
982,7933 DAFFODIL WAY,CITRUS HEIGHTS,95610,CA,3,2,1216,Residential,Thu May 15 00:00:00 EDT 2008,235000,38.708824,-121.256803
983,8304 RED FOX WAY,ELK GROVE,95758,CA,4,2,1685,Residential,Thu May 15 00:00:00 EDT 2008,235301,38.417000,-121.397424


### 2. Predict Price

We are going to predict `price` from available information.

#### 2.1 What is the Target Variable? Why?

In [7]:
# ANSWER
# price

#### 2.2 List all possible variables which might be Independent/Predictor variable.

In [8]:
# ANSWER
# sq__ft, beds, baths

#### 2.3 Find correlation between variables

Find correlation between variables. Which is the best predictor? Why? State your reason.

In [9]:
# ANSWER

#data <- dataframe name
house_csv.corr() #shows linear r/s between 2 variables (-ve inverse relationship, +ve positive relationship)

,zip,beds,baths,sq__ft,price,latitude,longitude
zip,1.000000,-0.072673,-0.103332,-0.101560,-0.219591,-0.134134,-0.562983
beds,-0.072673,1.000000,0.654746,0.695710,0.441988,-0.107440,0.056713
baths,-0.103332,0.654746,1.000000,0.724631,0.501820,-0.081249,0.108081
sq__ft,-0.101560,0.695710,0.724631,1.000000,0.693708,-0.103287,0.135809
price,-0.219591,0.441988,0.501820,0.693708,1.000000,0.058475,0.278312
latitude,-0.134134,-0.107440,-0.081249,-0.103287,0.058475,1.000000,0.344431
longitude,-0.562983,0.056713,0.108081,0.135809,0.278312,0.344431,1.000000


#### 2.4 Find Coefficient and Intercept using Linear Regression

Using LinearRegression of sklearn.linear_model package find Coefficient and Intercept.

Create an instance of LinearRegression.

Explore following methods:

- fit
- predict
- score

In [10]:
from sklearn.linear_model import LinearRegression

##### 2.4.1 Fit predictor and target variables using linear regression 

In [11]:
# ANSWER

X = house_csv['sq__ft'].values.reshape(-1,1) #.values.reshape(-1,1) reshapes the pandas series to a 2D dataframe
Y = house_csv['price'] # 
#reshape to a (985, 1) array # -1 up there can change to 985

In [12]:
X.shape

(814, 1)

In [13]:
#fit the model
lm = LinearRegression(n_jobs=1)
lm.fit(X,Y) 
# .fit calculate the optimal values of the weights 𝑏₀ and 𝑏₁, using the existing input and output (x and y) as the arguments


LinearRegression(n_jobs=1)

##### 2.4.2 Find R Score

Find $R^2$ Using ```score``` method of Linear Regression.

In [14]:
# ANSWER
lm.score(X, Y)

0.4812307133326901

### 3. Splitting Data

In [15]:
from sklearn.model_selection import train_test_split

#### 3.1 Create training and testing subsets

Hint: Using `train_test_split` Library.

```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
```

In [16]:
# ANSWER
## Create training and testing subsets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

#### 3.2 Check Shape, Sample of Test Train Data

In [17]:
# ANSWER
## Check training/test data
X_train.shape

(545, 1)

In [18]:
X_test.shape

(269, 1)

#### 3.3 Using Linear Regression Find The Score

1. Fit model using X_train, y_train
2. Find score using X_test, y_test

In [19]:
# ANSWER
lm.fit(X_train, Y_train)

LinearRegression(n_jobs=1)

In [20]:
# ANSWER
lm.score(X_test, Y_test) #r^2 of sq ft and price 
#[r^2 = output = % of variability can be explained with sqft e.g. can only explain 44% of increase in price]

0.44917703281062393

### 4. Predict House Price

Let's assume we have information of following house:

- street:	1140 EDMONTON DR
- city:	SACRAMENTO
- zip:	95833
- state:	CA
- beds:	3
- baths:	2
- sq__ft:	1204
- type:	Residential

**Predict the price of this house using linear regression model.**

In [28]:
# ANSWER

X = house_csv['sq__ft'].values.reshape(-1,1) 
Y = house_csv['price'] 

model1 = LinearRegression().fit(X,Y)

pred = [[1204]]
y_pred = model1.predict(pred)
y_pred

#write own dataframe and then predict

array([180971.28382945])

#### Find the error

In [29]:
# Predict price from X_test

#find error here

# Predict price from X_test

y_pred = model1.predict(X_test)

y_pred[:10]

from sklearn.metrics import mean_squared_error

mean_squared_error(Y_test, y_pred, squared=True)



6863202480.981789

In [30]:
pred_price_model = LinearRegression().fit(X_test,Y_test) #use x & y test data into the model to train it
pred_price = pred_price_model.predict(X) # pass X through model to get predicted price
pred_price

array([130232.97874597, 172745.34018695, 125095.53325462, 132287.95694252,
       125223.9693919 , 166965.71400917, 164653.86353806, 174029.70155979,
       143718.77316078, 170048.18130399, 139608.8167677 , 188414.54893558,
       134728.24355091, 153865.22800621, 154122.10028078, 168506.94765658,
       131260.46784425, 124967.09711733,  98380.81669956, 197019.77013361,
       166451.96946004, 193551.99442694, 182121.17820867, 228486.62376816,
       138581.32766943, 162598.88534152, 146544.36818103, 166580.40559732,
       200102.23742842, 183148.66730694, 156305.51461461, 170818.79812769,
       200102.23742842, 166195.09718547, 156305.51461461, 204982.81064521,
       161828.26851782, 211918.36205854, 170048.18130399, 120471.8323124 ,
       190341.09099484, 177882.7856783 , 158488.92894843, 156819.25916374,
       226688.51784619, 166708.8417346 , 225789.4648852 , 273953.01636667,
       235550.61131878, 212560.54274496, 175442.49906991, 227073.82625804,
       210762.43682298, 2



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



